In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - plot_dmod_posterior.ipynb
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Make a grid plot of the distance modulus posterior
'''

__author__ = "James Lane"

In [ ]:
### Imports

# Basic
import os, sys, pdb, copy
import numpy as np
from astropy import units as apu

# Matplotlib and plotting 
import matplotlib
import matplotlib.pyplot as plt

# mwdust, isodist stuff, galpy stuff
from isodist import FEH2Z, Z2FEH
from galpy.util import coords
from galpy import orbit

# Project specific
sys.path.insert(0,'../../src/')
from ges_mass import mass as pmass
from ges_mass import densprofiles as pdens
from ges_mass import util as putil
from ges_mass import plot as pplot

### Notebook setup

%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Keywords, Pathing, Loading, Data Preparation

In [ ]:
# %load ../../src/nb_modules/keywords_pathing_loading_data_prep.py
## Keywords
cdict = putil.load_config_to_dict()
keywords = ['BASE_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','NDMOD',
            'DMOD_MIN','DMOD_MAX','LOGG_MIN','LOGG_MAX','FEH_MIN','FEH_MAX',
            'FEH_MIN_GSE','FEH_MAX_GSE','DF_VERSION','KSF_VERSION','NPROCS',
            'RO','VO','ZO']
base_dir,apogee_dr,apogee_results_vers,gaia_dr,ndmod,dmod_min,dmod_max,\
    logg_min,logg_max,feh_min,feh_max,feh_min_gse,feh_max_gse,df_version,\
    ksf_version,nprocs,ro,vo,zo = putil.parse_config_dict(cdict,keywords)
logg_range = [logg_min,logg_max]
feh_range = [feh_min,feh_max]
feh_range_gse = [feh_min_gse,feh_max_gse]
feh_range_all = [feh_min,feh_max_gse]
# feh_range_fit = copy.deepcopy( # Need to choose here


## Pathing
fit_paths = putil.prepare_paths(base_dir,apogee_dr,apogee_results_vers,gaia_dr,
                                df_version,ksf_version)
data_dir,version_dir,ga_dir,gap_dir,df_dir,ksf_dir,fit_dir = fit_paths

## Filenames
fit_filenames = putil.prepare_filenames(ga_dir,gap_dir,feh_range_gse)
apogee_SF_filename,apogee_effSF_filename,apogee_effSF_mask_filename,\
    iso_grid_filename,clean_kinematics_filename = fit_filenames

## File loading and data preparation
fit_stuff,other_stuff = putil.prepare_fitting(fit_filenames,
    [ndmod,dmod_min,dmod_max], ro,zo,return_other=True)
apogee_effSF_mask,dmap,iso_grid,jkmins,dmods,ds,effsel_grid,apof,\
    allstar_nomask,orbs_nomask = fit_stuff
Rgrid,phigrid,zgrid = effsel_grid

# ## Load the distribution functions
# df_filename = df_dir+'dfs.pkl'
# betas = [0.3,0.8]
# dfs = putil.load_distribution_functions(df_filename, betas)

In [ ]:
fig_dir = './fig/'

### Global Parameters

In [ ]:
# %load ../../src/nb_modules/global_fitting_params.py
## general kwargs
verbose = True

## HaloFit kwargs (ordering follows HaloFit.__init__)
# allstar and orbs loaded in prep cell
init = None
init_type = 'ML'
# fit_type provided at runtime
mask_disk = True
mask_halo = True
# densfunc, selec provided at runtime
# effsel, effsel_grid, effsel_mask, dmods loaded in prep cell
nwalkers = 100
nit = int(2e3)
ncut = int(1e3)
# usr_log_prior provided at runtime
n_mass = 5000 # int(nwalkers*(nit-ncut))
int_r_range = [2.,70.]
iso = None # Will read from iso_grid_filename
# iso_filename, jkmins loaded in prep cell
# feh_range provided at runtime
# logg_range loaded in config cell
# fit_dir, gap_dir, ksf_dir loaded in prep cell
# version provided at runtime
# ro, vo, zo loaded in config cell

hf_kwargs = {## HaloFit parameters
             'allstar':allstar_nomask,
             'orbs':orbs_nomask,
             'init':init,
             'init_type':init_type,
             # 'fit_type':fit_type, # provided at runtime
             'mask_disk':mask_disk,
             'mask_halo':mask_halo,
             ## _HaloFit parameters
             # 'densfunc':densfunc, # provided at runtime
             # 'selec':selec, # provided at runtime
             'effsel':apof,
             'effsel_mask':apogee_effSF_mask,
             'effsel_grid':effsel_grid,
             'dmods':dmods,
             'nwalkers':nwalkers,
             'nit':nit,
             'ncut':ncut,
             # 'usr_log_prior':usr_log_prior, # provided at runtime
             'n_mass':n_mass,
             'int_r_range':int_r_range,
             'iso':iso,
             'iso_filename':iso_grid_filename,
             'jkmins':jkmins,
             # 'feh_range':feh_range, # provided at runtime
             'logg_range':logg_range,
             'fit_dir':fit_dir,
             'gap_dir':gap_dir,
             'ksf_dir':ksf_dir,
             # 'version':version, # provided at runtime
             'verbose':verbose,
             'ro':ro,
             'vo':vo,
             'zo':zo}

## pmass.fit() function kwargs
# nprocs set in config file
force_fit = True
mle_init = True
just_mle = False
return_walkers = True
optimizer_method = 'Powell'
mass_int_type = 'spherical_grid'
batch_masses = True
make_ml_aic_bic = True
calculate_masses = True
post_optimization = True
mcmc_diagnostic = True

fit_kwargs = {# 'nprocs':nprocs, # Normally given at runtime 
              'force_fit':force_fit,
              'mle_init':mle_init,
              'just_mle':just_mle,
              'return_walkers':return_walkers,
              'optimizer_method':optimizer_method,
              'mass_int_type':mass_int_type,
              'batch_masses':batch_masses,
              'make_ml_aic_bic':make_ml_aic_bic,
              'calculate_masses':calculate_masses,
              'post_optimization':post_optimization,
              'mcmc_diagnostic':mcmc_diagnostic,
              }

## Make the plot for GS/E and halo data

In [ ]:
# Arrays to map density profile to the version of the fit to show for
# that density profile
densfuncs = [pdens.triaxial_single_angle_zvecpa,
             pdens.triaxial_single_angle_zvecpa_plusexpdisk,
             pdens.triaxial_single_cutoff_zvecpa,
             pdens.triaxial_single_cutoff_zvecpa_plusexpdisk,
             pdens.triaxial_broken_angle_zvecpa,
             pdens.triaxial_broken_angle_zvecpa_plusexpdisk,
             pdens.triaxial_double_broken_angle_zvecpa,
             pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk]

# Shorthand for if they're all the same version
_version = '100w_2e3n'
versions = {'gse':[_version]*len(densfuncs),
            'all':[_version]*len(densfuncs)
            }
# versions = {'gse':['100w_2e3n',
#                    '100w_2e3n',
#                    '100w_2e3n',
#                    '100w_2e3n',
#                    '100w_2e3n',
#                    '100w_2e3n',
#                    '100w_2e3n',
#                    '100w_2e3n',
#             ],
#             'all':['100w_2e3n',
#                    '100w_2e3n',
#                    '100w_2e3n',
#                    '100w_2e3n',
#                    '100w_2e3n',
#                    '100w_2e3n',
#                    '100w_2e3n',
#                    '100w_2e3n',
#                ]
#            }

selecs = ['eLz','AD','JRLz',None]
selecs_names = [r'$e-L_\mathrm{z}$',r'AD',r'$\sqrt{J_\mathrm{R}}-L_\mathrm{z}$',
                'Halo']
densfunc_labels = ['SPL','SPL+D','SC','SC+D','BPL','BPL+D','DBPL','DBPL+D']

In [ ]:
# Plotting and HaloFit kwargs
n_bins = 10
nrand = 3
verbose = False
hf_kwargs['verbose'] = verbose

# Fontsize and formatting kwargs
ylabel_fs = 12
xlabel_fs = 12
posterior_lw = 1
hist_lw = 1.5
hist_err_capsize = 2
legend_fs = 6
ticklabel_fs = 10
annotate_fs = 10
columnwidth = 244./72.27 # Only one column. In inches, from pt
textwidth = 508./72.27 # Whole page. In inches, from pt


# Make figure
fig = plt.figure(figsize=(textwidth,2.5))
axs = fig.subplots(nrows=1, ncols=4).flatten()

colors = ['Red','DarkOrange','DodgerBlue','Navy']
linestyles = ['solid','dashed']

for i in range(4):
    
    # if i > 0: continue

    if i == 3:
        _feh_range = copy.deepcopy(feh_range_all)
        _fit_type = 'all'
    else:
        _feh_range = copy.deepcopy(feh_range_gse)
        _fit_type = 'gse'
    
    # Load the results through the HaloFit class
    _hf = pmass.HaloFit(densfunc=densfuncs[0], fit_type=_fit_type, 
                       version=versions[_fit_type][0], selec=selecs[i], 
                       feh_range=_feh_range, **hf_kwargs)
    orbs = _hf.orbs
    dm = 5*np.log10(orbs.dist(use_physical=True).to(apu.kpc).value)+10
    hist,bin_edges = np.histogram(dm, bins=n_bins, 
        range=(np.min(_hf.dmods), np.max(_hf.dmods)) )
    bin_cents = (bin_edges[1:]+bin_edges[:-1])/2
    bin_err = np.sqrt(hist)
    dens_norm = np.sum(hist)*(bin_edges[1]-bin_edges[0])

    axs[i].hist(dm, bins=n_bins, histtype='step', range=(np.min(_hf.dmods),
                np.max(_hf.dmods)), density=True, color='Black', zorder=2, 
                linewidth=hist_lw)
    axs[i].errorbar(bin_cents, hist/dens_norm, yerr=bin_err/dens_norm, 
        fmt='', capsize=hist_err_capsize, color='None', ecolor='Black', 
        zorder=2, alpha=0.5)

    pdstd = np.zeros((len(densfuncs),len(_hf.dmods)))

    for j in range(4):
        indx = int(2*i+j)
        for k in range(2):
            
            indx = int(2*j+k)
            # Load the results through the HaloFit class
            hf = pmass.HaloFit(densfunc=densfuncs[indx], fit_type=_fit_type, 
                               version=versions[_fit_type][indx], selec=selecs[i], 
                               feh_range=_feh_range, **hf_kwargs)
            hf.get_results()
            
            # print('Calculating p(distmod|model)')
            if nrand is not None:
                nrand = int(nrand)
                rnp = np.random.default_rng()
                randind = rnp.integers(0,hf.samples.shape[0],size=nrand)
                samples_in = hf.samples[randind]
            else:
                samples_in = hf.samples[:]
            Rgrid,phigrid,zgrid = hf.get_effsel_list()
            pd,_,_ = pmass.pdistmod_sample(hf.densfunc, samples_in, 
                hf.get_fit_effsel(), hf.Rgrid, hf.phigrid, hf.zgrid, hf.dmods, 
                return_rate=True, verbose=hf.verbose)
            
            pdstd[indx,:] = np.std(pd,axis=0)
            pdmed = np.median(pd,axis=0)
            if k == 0:
                axs[i].plot(hf.dmods, pdmed, zorder=3, color=colors[j], 
                            linewidth=posterior_lw, linestyle=linestyles[k], 
                            label=densfunc_labels[indx])
            else:
                axs[i].plot(hf.dmods, pdmed, zorder=3, color=colors[j], 
                            linewidth=posterior_lw, linestyle=linestyles[k])
    
    pdstd_mean = np.mean(pdstd,axis=0)
    pdstd_height = np.ones_like(_hf.dmods)*0.325
    axs[i].plot(_hf.dmods, pdstd_height, color='Black', linewidth=0.5)
    axs[i].fill_between(_hf.dmods, pdstd_height+pdstd_mean, 
                        pdstd_height-pdstd_mean, color='Black', alpha=0.25,
                        linewidth=0.25)

    if i == 0:
        axs[i].set_ylabel(r'p($\mu$)',fontsize=ylabel_fs)
    else:
        axs[i].tick_params(labelleft=False)
    axs[i].tick_params(which='both',labelsize=ticklabel_fs)
    axs[i].set_xlabel(r'$\mu$',fontsize=xlabel_fs)
    axs[i].set_xlim(np.min(_hf.dmods)-1, np.max(_hf.dmods)+1)
    axs[i].set_ylim(-0.01, 0.35)
    axs[i].text(0.1, 0.8, selecs_names[i], transform=axs[i].transAxes,
                fontsize=annotate_fs)
    if i == 0:
        axs[i].legend(loc='center left', fontsize=legend_fs, frameon=False)

plt.tight_layout()
fig.subplots_adjust(wspace=0.05,left=0.075,right=0.95)#,bottom=0.175,top=0.95)
fig.savefig(fig_dir+'pdistmod.pdf')
fig.show()